In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import matplotlib.pyplot as plt 
import numpy as np
from functools import partial
import json
from glob import glob

import cv2
import os
import random

import csv
import pandas as pd
import shutil

from scipy import interpolate
#import nia22
#from nia22.generator import *

In [3]:
def get_converter(field:str):
    if field == "inst":
        keys = ["S", "T", "L", "V", "M"]
    elif field == "condition":
        keys = ["F", "S", "D", "A", "N"]
    elif field == "posture":
        keys = ["S", "D", "P", "L", "F", 
                "C", "H", "E", "T", "U"]
    else:
        raise ValueError(field)

    return partial(_concat, keys=keys)

def _concat(value, keys=None):
    """concatenate strings"""
    code =""
    for s in keys:
        if s in value:
            code += s
        else:
            code += "X"
    return code

# get total file size
def get_total_file_size(wdir, pattern, flist=False):
    all_files = glob(wdir+ pattern)
    all_files.sort()
    nfiles = len(all_files)
    
    print(f"found {nfiles} files")
    
    file_sizes_Bytes = [os.path.getsize(cc) for cc in all_files]
    tsize_inMB = np.sum(file_sizes_Bytes) / 1024 / 1024
    
    all_file_names = [al.split("processing/")[1] for al in all_files]
    if tsize_inMB > 1024:
        if flist:
            return nfiles, f"{tsize_inMB/1024:.2f}GB", all_file_names
        else:
            return nfiles, f"{tsize_inMB/1024:.2f}GB"
    else:
        if flist:
            return nfiles, f"{tsize_inMB:.2f}MB", all_file_names
        else:
            return nfiles, f"{tsize_inMB:.2f}MB"

def change_age(age):
    if age.startswith("1"):
        age = 10
    elif age.startswith("2"):
        age = 20
    elif age.startswith("3"):
        age = 30
    elif age.startswith("4"):
        age = 40
    elif age.startswith("5"):
        age = 50
    elif age.startswith("6"):
        age = 60
    return age

In [22]:
import pandas as pd
import csv, json
from datetime import datetime

import numpy as np

xlsx = pd.read_excel('/home/di/Work/data/NIA2022/메타데이터.xlsx')

xlsx = list(xlsx.values)



In [26]:
concat_inst = get_converter("inst")
concat_condition = get_converter("condition")
concat_posture = get_converter("posture")

In [177]:
# Loop over all csv files
for date in [2,4,5,8,9,10,11,12,16,17,18,19,22,23,25][10:]:
    wdir = f'/home/di/Work/data/NIA2022/08{date:02d}/processing/S1/'
    #'/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/'

    video_list = glob(wdir+"0??/T1/*/RGB/*_rgb_*.mp4")
    video_list.sort()
    
    for ff in video_list:
        dir_names = ["Eye-tracker/", "FaceAngle/", 
                     "CamAngle/", "DistCam2Face/", 
                     "DistDisp2Face/"]
        file_names = ["_point_", "_head_", "_cam_", 
                      "_dcam_", "_ddisp_"]

        cap = cv2.VideoCapture(ff)
        nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        nlines = []
        for (dd, fn) in zip(dir_names, file_names):
            fnew = ff.replace("RGB/", dd)
            fnew = fnew.replace("_rgb_", fn)
            fnew = fnew.replace(".mp4", ".csv")
            if os.path.isfile(fnew):
                if os.path.getsize(fnew) < 40:
                    shutil.move(fnew, )
                    #print("small file", fnew)

small file /home/di/Work/data/NIA2022/0818/processing/S1/022/T1/Smartphone/FaceAngle/NIA22EYE_S1_022_T1_S01_S_head_F_D_T.csv
small file /home/di/Work/data/NIA2022/0818/processing/S1/022/T1/Smartphone/FaceAngle/NIA22EYE_S1_022_T1_S01_S_head_F_H_T.csv
small file /home/di/Work/data/NIA2022/0818/processing/S1/022/T1/Tablet/FaceAngle/NIA22EYE_S1_022_T1_S01_T_head_D_C_R.csv
small file /home/di/Work/data/NIA2022/0818/processing/S1/022/T1/Tablet/FaceAngle/NIA22EYE_S1_022_T1_S01_T_head_D_D_L.csv
small file /home/di/Work/data/NIA2022/0818/processing/S1/022/T1/Tablet/FaceAngle/NIA22EYE_S1_022_T1_S01_T_head_D_D_R.csv
small file /home/di/Work/data/NIA2022/0818/processing/S1/022/T1/Tablet/FaceAngle/NIA22EYE_S1_022_T1_S01_T_head_D_U_L.csv
small file /home/di/Work/data/NIA2022/0818/processing/S1/022/T1/Tablet/FaceAngle/NIA22EYE_S1_022_T1_S01_T_head_D_U_R.csv
small file /home/di/Work/data/NIA2022/0818/processing/S1/022/T1/Tablet/FaceAngle/NIA22EYE_S1_022_T1_S01_T_head_F_C_L.csv
small file /home/di/Work

small file /home/di/Work/data/NIA2022/0822/processing/S1/026/T1/Laptop/FaceAngle/NIA22EYE_S1_026_T1_S06_L_head_A_T_T.csv
small file /home/di/Work/data/NIA2022/0822/processing/S1/026/T1/Laptop/FaceAngle/NIA22EYE_S1_026_T1_S06_L_head_D_T_T.csv
small file /home/di/Work/data/NIA2022/0822/processing/S1/026/T1/Laptop/FaceAngle/NIA22EYE_S1_026_T1_S06_L_head_F_T_T.csv
small file /home/di/Work/data/NIA2022/0822/processing/S1/026/T1/Laptop/FaceAngle/NIA22EYE_S1_026_T1_S06_L_head_N_T_T.csv
small file /home/di/Work/data/NIA2022/0822/processing/S1/026/T1/Laptop/FaceAngle/NIA22EYE_S1_026_T1_S07_L_head_A_T_T.csv
small file /home/di/Work/data/NIA2022/0822/processing/S1/026/T1/Laptop/FaceAngle/NIA22EYE_S1_026_T1_S07_L_head_D_T_T.csv
small file /home/di/Work/data/NIA2022/0822/processing/S1/026/T1/Laptop/FaceAngle/NIA22EYE_S1_026_T1_S07_L_head_F_T_T.csv
small file /home/di/Work/data/NIA2022/0822/processing/S1/026/T1/Laptop/FaceAngle/NIA22EYE_S1_026_T1_S07_L_head_N_T_T.csv
small file /home/di/Work/data/NI

small file /home/di/Work/data/NIA2022/0822/processing/S1/027/T1/Tablet/FaceAngle/NIA22EYE_S1_027_T1_S10_T_head_A_T_L.csv
small file /home/di/Work/data/NIA2022/0822/processing/S1/027/T1/Tablet/FaceAngle/NIA22EYE_S1_027_T1_S10_T_head_A_T_R.csv
small file /home/di/Work/data/NIA2022/0822/processing/S1/027/T1/Tablet/FaceAngle/NIA22EYE_S1_027_T1_S10_T_head_A_T_T.csv
small file /home/di/Work/data/NIA2022/0822/processing/S1/028/T1/Laptop/FaceAngle/NIA22EYE_S1_028_T1_S08_L_head_A_T_T.csv
small file /home/di/Work/data/NIA2022/0822/processing/S1/028/T1/Laptop/FaceAngle/NIA22EYE_S1_028_T1_S08_L_head_D_T_T.csv
small file /home/di/Work/data/NIA2022/0822/processing/S1/028/T1/Laptop/FaceAngle/NIA22EYE_S1_028_T1_S08_L_head_F_T_T.csv
small file /home/di/Work/data/NIA2022/0822/processing/S1/028/T1/Laptop/FaceAngle/NIA22EYE_S1_028_T1_S08_L_head_N_T_T.csv
small file /home/di/Work/data/NIA2022/0822/processing/S1/028/T1/Laptop/FaceAngle/NIA22EYE_S1_028_T1_S09_L_head_A_T_T.csv
small file /home/di/Work/data/NI

FileNotFoundError: [Errno 2] No such file or directory: '/home/di/Work/data/NIA2022/0823/processing/S1/030/T1/Tablet/CamAngle/NIA22EYE_S1_030_T1_S09_T_cam_D_F_T.csv'

In [174]:
fnew

'/home/di/Work/data/NIA2022/0822/processing/S1/028/T1/VehicleLCD/DistDisp2Face/NIA22EYE_S1_028_T1_S10_V_ddisp_S_E_T.csv'

In [173]:
os.path.getsize(fnew)

1209

In [162]:
for date in [2,4,5,8,9,10,11,12,16,17,18,19,22,23,25]:
    wdir = f'/home/di/Work/data/NIA2022/08{date:02d}/processing/S1/'
    #'/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/'

    video_list = glob(wdir+"0??/T1/*/RGB/*_rgb_*.mp4")
    #glob(wdir+"0??/T1/*/*/*.csv")
    all_csv = glob(wdir+"0??/T1/*/*/*.csv")
    all_csv.sort()
    video_list.sort()
    print(len(video_list))

    # .mp4 파일명 수정
    # .csv도 필요하면 적용
    # mp4 먼저 고치고 나머지 처리하면 csv 파일 이름 변경할 필요 없음. 
    for fncsv in video_list:
        fnew = fncsv.replace("_K_", "_T_")
        if fnew != fncsv:
            print("ERROR")
            shutil.move(fncsv, fnew)
    
    for fncsv in all_csv:
        fnew = fncsv.replace("_K_", "_T_")
        if fnew != fncsv:
            print("ERROR")
            shutil.move(fncsv, fnew)

904
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR


ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERRO

ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
453


In [161]:
len(all_csv)

0

In [181]:
for date in [2,4,5,8,9,10,11,12,16,17,18,19,22,23,25][-3:-2]:
    wdir = f'/home/di/Work/data/NIA2022/08{date:02d}/processing/S1/'
    #'/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/'

    video_list = glob(wdir+"0??/T1/*/RGB/*_rgb_*.mp4")
    video_list.sort()
    
    for ff in video_list:
        dir_names = ["Eye-tracker/", "FaceAngle/", 
                     "CamAngle/", "DistCam2Face/", 
                     "DistDisp2Face/"]
        file_names = ["_point_", "_head_", "_cam_", 
                      "_dcam_", "_ddisp_"]

        cap = cv2.VideoCapture(ff)
        nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        nlines = []
        for (dd, fn) in zip(dir_names, file_names):
            fnew = ff.replace("RGB/", dd)
            fnew = fnew.replace("_rgb_", fn)
            fnew = fnew.replace(".mp4", ".csv")

            f = open(fnew, "r")
            nlines.append(len(f.readlines()))
            f.close()

        print(nframes, nlines)
    print("_________________\n\n")

330 [331, 331, 331, 331, 331]
327 [328, 328, 328, 328, 328]
328 [329, 329, 329, 329, 329]
327 [328, 328, 328, 328, 328]
328 [329, 329, 329, 329, 329]
329 [330, 330, 330, 330, 330]
327 [328, 328, 328, 328, 328]
328 [329, 329, 329, 329, 329]
327 [328, 328, 328, 328, 328]
327 [328, 328, 328, 328, 328]
328 [329, 329, 329, 329, 329]
329 [330, 330, 330, 330, 330]
328 [329, 329, 329, 329, 329]
325 [326, 326, 326, 326, 326]
329 [330, 330, 330, 330, 330]
327 [328, 328, 328, 328, 328]
324 [325, 325, 325, 325, 325]
328 [329, 329, 329, 329, 329]
328 [329, 329, 329, 329, 329]
328 [329, 329, 329, 329, 329]
328 [329, 329, 329, 329, 329]
326 [327, 327, 327, 327, 327]
328 [329, 329, 329, 329, 329]
328 [329, 329, 329, 329, 329]
327 [328, 328, 328, 328, 328]
327 [328, 328, 328, 328, 328]
328 [329, 329, 329, 329, 329]
327 [328, 328, 328, 328, 328]
328 [329, 329, 329, 329, 329]
327 [328, 328, 328, 328, 328]
328 [329, 329, 329, 329, 329]
327 [328, 328, 328, 328, 328]
326 [327, 327, 327, 327, 327]
329 [330, 

301 [302, 302, 302, 302, 302]
302 [303, 303, 303, 303, 303]
300 [301, 301, 301, 301, 301]
300 [301, 301, 301, 301, 301]
301 [302, 302, 302, 302, 302]
301 [302, 302, 302, 302, 302]
300 [301, 301, 301, 301, 301]
302 [303, 303, 303, 303, 303]
302 [303, 303, 303, 303, 303]
304 [305, 305, 305, 305, 305]
306 [307, 307, 307, 307, 307]
307 [308, 308, 308, 308, 308]
310 [311, 311, 311, 311, 311]
307 [308, 308, 308, 308, 308]
307 [308, 308, 308, 308, 308]
308 [309, 309, 309, 309, 309]
306 [307, 307, 307, 307, 307]
302 [303, 303, 303, 303, 303]
307 [308, 308, 308, 308, 308]
306 [307, 307, 307, 307, 307]
304 [305, 305, 305, 305, 305]
303 [304, 304, 304, 304, 304]
303 [304, 304, 304, 304, 304]
302 [303, 303, 303, 303, 303]
303 [304, 304, 304, 304, 304]
300 [301, 301, 301, 301, 301]
311 [312, 312, 312, 312, 312]
309 [310, 310, 310, 310, 310]
309 [310, 310, 310, 310, 310]
301 [302, 302, 302, 302, 302]
301 [302, 302, 302, 302, 302]
301 [302, 302, 302, 302, 302]
301 [302, 302, 302, 302, 302]
301 [302, 

306 [307, 307, 307, 307, 307]
307 [308, 308, 308, 308, 308]
304 [305, 305, 305, 305, 305]
313 [314, 314, 314, 314, 314]
305 [306, 306, 306, 306, 306]
306 [307, 307, 307, 307, 307]
306 [307, 307, 307, 307, 307]
305 [306, 306, 306, 306, 306]
303 [304, 304, 304, 304, 304]
297 [298, 298, 298, 298, 298]
258 [259, 259, 259, 259, 259]
305 [306, 306, 306, 306, 306]
246 [247, 247, 247, 247, 247]
300 [301, 301, 301, 301, 301]
305 [306, 306, 306, 306, 306]
306 [307, 307, 307, 307, 307]
306 [307, 307, 307, 307, 307]
306 [307, 307, 307, 307, 307]
306 [307, 307, 307, 307, 307]
310 [311, 311, 311, 311, 311]
306 [307, 307, 307, 307, 307]
308 [309, 309, 309, 309, 309]
309 [310, 310, 310, 310, 310]
312 [313, 313, 313, 313, 313]
305 [306, 306, 306, 306, 306]
305 [306, 306, 306, 306, 306]
306 [307, 307, 307, 307, 307]
311 [312, 312, 312, 312, 312]
305 [306, 306, 306, 306, 306]
304 [305, 305, 305, 305, 305]
306 [307, 307, 307, 307, 307]
304 [305, 305, 305, 305, 305]
305 [306, 306, 306, 306, 306]
308 [309, 

312 [313, 313, 313, 313, 313]
313 [314, 314, 314, 314, 314]
313 [314, 314, 314, 314, 314]
307 [308, 308, 308, 308, 308]
311 [312, 312, 312, 312, 312]
309 [310, 310, 310, 310, 310]
306 [307, 307, 307, 307, 307]
306 [307, 307, 307, 307, 307]
305 [306, 306, 306, 306, 306]
306 [307, 307, 307, 307, 307]
307 [308, 308, 308, 308, 308]
305 [306, 306, 306, 306, 306]
245 [246, 246, 246, 246, 246]
245 [246, 246, 246, 246, 246]
305 [306, 306, 306, 306, 306]
174 [175, 175, 175, 175, 175]
153 [154, 154, 154, 154, 154]
154 [155, 155, 155, 155, 155]
306 [307, 307, 307, 307, 307]
305 [306, 306, 306, 306, 306]
305 [306, 306, 306, 306, 306]
307 [308, 308, 308, 308, 308]
310 [311, 311, 311, 311, 311]
308 [309, 309, 309, 309, 309]
312 [313, 313, 313, 313, 313]
308 [309, 309, 309, 309, 309]
308 [309, 309, 309, 309, 309]
305 [306, 306, 306, 306, 306]
310 [311, 311, 311, 311, 311]
308 [309, 309, 309, 309, 309]
305 [306, 306, 306, 306, 306]
308 [309, 309, 309, 309, 309]
305 [306, 306, 306, 306, 306]
305 [306, 

In [180]:
for ff in video_list:
    dir_names = ["Eye-tracker/", "FaceAngle/", "CamAngle/", "DistCam2Face/", "DistDisp2Face/"]
    file_names = ["_point_", "_head_", "_cam_", "_dcam_", "_ddisp_"]

    cap = cv2.VideoCapture(ff)
    nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    #ff = ff.replace("_K_", "_T_")
    #ff = ff.replace("_K_R.csv", "_K_R.csv")
    
    #print(nframes)

    nlines = []
    for (dd, fn) in zip(dir_names, file_names):
        fnew = ff.replace("RGB/", dd)
        fnew = fnew.replace("_rgb_", fn)
        fnew = fnew.replace(".mp4", ".csv")
    
        ### csv 길이 맞추기
        #for ff in flist:
        f = open(fnew, "r")
        nlines = len(f.readlines())
        if nlines != nframes +1:
            f.seek(0)
            f.readline()
            rrr = csv.reader(f, quoting=csv.QUOTE_NONNUMERIC)
            all_lines = []
            for rr in rrr:
                all_lines.append(rr)
            f.close()
            
            arr = np.array(all_lines).astype(float) #####
            nn = len(arr)
            x = np.arange(nn)

            inter = interpolate.interp1d(x, arr[:,0])
            x_new = np.linspace(0, nn-1, nframes)    
            c1 = inter(x_new)

            #new_fn = ff.replace("S1/", "S1_new/")

            if "Dist" in dd:
                with open(fnew, 'w') as f:
                    f.write("distance\n")
                    for v1 in c1:
                        f.write(f"{v1}\n")
            
            else:
                inter = interpolate.interp1d(x, arr[:,1])
                c2 = inter(x_new)

            if dd == "Eye-tracker/":
                with open(fnew, 'w') as f:
                    f.write("[point] x   y\n")
                    for v1,v2 in zip(c1,c2):
                        f.write(f"{int(v1)}, {int(v2)}\n")

            if "Angle" in dd:
                inter = interpolate.interp1d(x, arr[:,2])
                c3 = inter(x_new)   

                with open(fnew, 'w') as f:
                    f.write(f"Roll Pitch Yaw\n")
                    for v1,v2,v3 in zip(c1,c2,c3):
                        f.write(f"{v1:.3f}, {v2:.3f}, {v3:.3f}\n")


In [61]:
xlsx[:-24]

[array(['김태훈', '남', 1, 'OB001', '27세 (1996년생)', '미착용', '미착용', '453개',
        Timestamp('2022-08-02 00:00:00'), nan], dtype=object),
 array(['윤지환', '남', 2, 'OB002', '27세 (1996년생)', '착용', '미착용', nan,
        Timestamp('2022-08-02 00:00:00'), nan], dtype=object),
 array(['김은서', '여', 3, 'OB003', '20세 (2003년생)', '착용', '미착용', nan,
        Timestamp('2022-08-03 00:00:00'), nan], dtype=object),
 array(['천정호', '남', 4, 'OB004', '56세 (1967년생)', '미착용', '미착용', nan,
        Timestamp('2022-08-03 00:00:00'), nan], dtype=object),
 array(['이경희', '여', 5, 'OB005', '50세 (1973년생)', '착용', '미착용', nan,
        Timestamp('2022-08-04 00:00:00'), nan], dtype=object),
 array(['이수미', '여', 6, 'OB006', '47세 (1976년생) ', '미착용', '미착용', nan,
        Timestamp('2022-08-04 00:00:00'), nan], dtype=object),
 array(['한민선', '여', 7, 'OB007', '25세 (1998년생) ', '착용', '미착용', nan,
        Timestamp('2022-08-04 00:00:00'), nan], dtype=object),
 array(['정혜은', '여', 8, 'OB008', '46세 (1977년생)', '미착용', '미착용', nan,
        Timestamp('202

In [75]:
nas_dir = ['/media/di/mx500_2tb2/NIA2022_2/', "/media/di/data/NIA2022/"][1]
#wdir = "/media/di/data/NIA2022/"

### 공통
loc = "S101"
inst = "STLVM"
condition = "FSDAN"
posture = "SDPLFCHETU"
###
for data in xlsx[:2]:
    clip_list_fn =[]
    
    gender, subject, medical, age, glasses, mask, ver, date, _ = data[1:10]
    date, hour = str(date).split(" ")
    hour = hour[:-3]
    
    ## Variables below are to be read from csv file ##
    username = "hoseung"
    email = "hschoi@dinsight.ai"
    glasses = "no" if glasses == "미착용" else "yes"
    cosmetic= "no"
    mask = "no" if mask == "미착용" else "yes"
    gender = "male" if gender == "남" else "여"
    
    rev = str(0)

    wdir = nas_dir + f'{"".join(date.split("-")[1:])}/processing/S1/{subject:03d}/T1/*/'
    
    n_point, tsize_point = get_total_file_size(wdir+"Eye-tracker/", "*.csv")
    n_head, tsize_head = get_total_file_size(wdir+"FaceAngle/", "*.csv")
    n_cam, tsize_cam = get_total_file_size(wdir+"CamAngle/", "*.csv") 
    n_dcam, tsize_d_cam = get_total_file_size(wdir+"DistCam2Face/", "*.csv") # was display pose
    n_disp, tsize_d_disp = get_total_file_size(wdir+"DistDisp2Face/", "*.csv")
    n_rgb, tsize_rgb, cl_rgb = get_total_file_size(wdir+"RGB/", "*_rgb_*.mp4", flist=True)
    n_ir, tsize_ir, cl_ir = get_total_file_size(wdir+"IR/", "*_ir_*.mp4", flist=True)
    clip_list_fn = cl_rgb + cl_ir
    ######################################
    
    ver  = n_cam
    
    x = {
        "id": {"subject":subject, 
               "medical":medical},
        "age":change_age(age), 
        "glasses":glasses, 
        "cosmetic":cosmetic, 
        "mask":mask, 
        "gender":gender,
        "ver":ver, 
        "date":date, 
        "loc":loc, 
        "inst":concat_inst(inst), 
        "condition":concat_condition(condition),
        "posture":concat_posture(posture),
        "rev":rev,
        "image": {"format": {"rgb": "mp4","ir": "mp4", "point":"csv", "head":"csv", 
                    #         "gaze":"csv", 
                             "cam":"csv",
                             "d_cam":"csv", "d_disp":"csv"},
                    "spec":{"rgb.w":1920, "rgb.h":1080, "ir.w":1920, "ir.h":1080, 
                             "point":"x,y",
                             "head":"deg,deg,deg",
                    #         "gaze":"yyyy-mm-dd,hh:mm:ss,deg,deg,deg",
                             "cam":"deg,deg,deg",
                             "d_cam":"cm",
                             "d_disp":"cm"},
                    "total_size":{"rgb":tsize_rgb, "ir":tsize_ir, "point":tsize_point, "head":tsize_head,
                    #              "gaze":tsize_gaze, 
                                  "cam":tsize_cam, "d_cam":tsize_d_cam, "d_disp":tsize_d_disp},
                    "total_count":{"rgb":n_rgb, "ir":n_ir, "point":n_point, "head":n_head,
                    #               "gaze":n_gaze,
                                   "cam":n_cam, "d_cam":n_dcam, "d_disp":n_disp}
                 },
        "task":{
            "name":"NIA_EYE",
            "created": date + ", " + hour, 
            "updated":"", 
            "username":username, 
            "email":email},
        #"filelist":clip_list_fn
    }

    # Dump json file
    fn = f"{subject:03d}.json"
    with open(fn, "w") as f:
        json.dump(x, f, indent=2, ensure_ascii=False)

found 452 files
found 452 files
found 452 files
found 452 files
found 452 files
found 452 files
found 126 files
found 452 files
found 452 files
found 452 files
found 452 files
found 452 files
found 452 files
found 125 files


In [123]:
pwd

'/home/di/Work/nia/nia'

In [7]:
def csv_to_arr(fn, skip_header=True):
    with open(fn, "r") as f:
        f.seek(0)
        if skip_header: f.readline()
        rrr = csv.reader(f, quoting=csv.QUOTE_NONNUMERIC)
        all_lines = []
        for rr in rrr:
            all_lines.append(rr)

    return np.array(all_lines).astype(float)

def matching_string(string, slist):
    for i, ss in enumerate(slist):
        if string in ss:
            return i

get_pose = lambda x : x.split("/")[-1][-9:]
get_fn   = lambda x : x.split("/")[-1]

In [72]:
date = 4
device=["Smartphone", "Tablet"][0]
wgood = f'/media/di/data/NIA2022/08{date:02d}/processing/S1/006/T1/'
wbad = f'/media/di/data/NIA2022/08{date:02d}/processing/S1/007/T1/'

fgood = glob(wgood + f"{device}/CamAngle/*.csv")

fbadtarget = glob(wbad + f"{device}/DistCam2Face/*.csv")
fbad  = glob(wbad  + f"{device}/CamAngle/*.csv")
fgood.sort()
fbad.sort()
fbadtarget.sort()

all_good_fn = [*map(get_fn, fgood)]
all_bad_fn =  [*map(get_fn, fbad)]
all_bad_target =  [*map(get_fn, fbadtarget)]

all_good_pose = [*map(get_pose, fgood)]
all_bad_pose =  [*map(get_pose, fbad)]
all_bad_target_pose =  [*map(get_pose, fbadtarget)]
all_common_pose = np.intersect1d(all_good_pose, all_bad_pose)

missing = []
for this_csv in all_bad_target:
    if this_csv.replace("_dcam_", "_cam_") not in all_bad_fn:
        missing.append(this_csv)

# missing = []
# for good in all_bad_target: # 같은 사람, 다른 기기여야함. 다른 사람이면 안 돼! 
#     if good not in all_bad_pose:
#         missing.append(good)

ind_arr = np.arange(len(all_common_pose))
for missing_fn in missing:
    good_missing = fgood[matching_string(missing_fn[-9:], fgood)]
    
    random.shuffle(ind_arr)
    
    ii = 0
    while True:
        # r,p,y 모두 90도 안에 들어올 때 까지 기준점을 랜덤으로 고름
        iorg = ind_arr[ii]
        comm_key = all_common_pose[iorg]
        good_origin = fgood[matching_string(comm_key, fgood)]
        bad_origin  = fbad[matching_string(comm_key, fbad)]
        
        arr0 = csv_to_arr(good_origin)
        arr_g = csv_to_arr(good_missing)
        arr0b = csv_to_arr(bad_origin)

        arr_bad_missing = arr0b - (arr0 - arr_g)
        
        if np.max(np.abs(arr_bad_missing)) < 180:
            break
        else:
            ii+=1

    # Save to a new file
    fn_bad_missing = wbad  + f"{device}/CamAngle/" + missing_fn.replace("_dcam_", "_cam_")

    with open(fn_bad_missing, 'w') as f:
        f.write(f"Roll Pitch Yaw\n")
        for v1,v2,v3 in arr_bad_missing:
            f.write(f"{v1:.3f}, {v2:.3f}, {v3:.3f}\n")

In [7]:
def point_to_degree(xpos, ypos, d_cam, d_disp, ):
    """transformation이 너무... 많은데..?? 
    """

In [9]:
# count the number of frames of the video
conditions = ["F", "S", "D", "A", "N"]
postures = ["S", "D", "P", "L", "F", "C", "H", "E", "T", "U"]
display = ["S", "T", "L", "V", "M"]

wdir = "./"

for fn_clip in clip_list:
    file_name = fn_clip.split("/")[-1]
    display, posture, condition, orientation = file_name.split(".mp4")[0].split("_")
    cap = cv2.VideoCapture(fn_clip)
    if not cap.isOpened():
        continue
    nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    #print(display, posture, condition, orientation)
    fake = Fake_generator(wdir, nframes, 1920, 1080, display, posture, condition, orientation)
    dist = 35 # 정하고 하는거 아님
    fake.gen_d_disp(dist)
    fake.gen_d_cam(dist)
    fake.gen_head()
    fake.gen_pose_d()
    fake.gen_gaze_eye_trakcer()
    fake.save_all()